In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
import os
sys.path.append('/content/drive/MyDrive')
sys.path.append('/content/drive/MyDrive/labs_cv')
os.chdir('/content/drive/MyDrive')

Imports

In [3]:
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms
import torch.nn as nn
from torchvision import models
import torch.nn.functional as F
import torch

import cv2

import pandas as pd
import scipy as sc
import numpy as np
import matplotlib.pyplot as plt

from collections import defaultdict
from tqdm import tqdm
import copy
import random

In [4]:
from bricks_dataset import BrickDataset

In [5]:
!unzip -u 'labs_cv/data.zip' -d '../../'

Archive:  labs_cv/data.zip


In [6]:
DATASET_TRAIN_CSV = '..//..//data//description//train.csv'
DATASET_VALIDATE_CSV = '..//..//data//description//test.csv'

Get the mean and std for train data

In [7]:
transform = transforms.Compose(
    [
        transforms.ToTensor()
    ]
)
dataset_train = BrickDataset(DATASET_TRAIN_CSV, "../", transform)
train_loader = DataLoader(dataset_train, batch_size=32)

In [8]:
def rotation(image):
  return TF.rotate(image, random.choice([0, 90, 180, 270]))

train_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        rotation
    ]
)

val_transform = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)

In [9]:
train_dataset = BrickDataset(DATASET_TRAIN_CSV, "../", train_transform)
val_dataset = BrickDataset(DATASET_VALIDATE_CSV, "../", val_transform)

In [10]:
class MetricMonitor:
    def __init__(self, float_precision=4):
      self.float_precision = float_precision

      self.metrics = None
      self.reset()

    def reset(self):
      self.metrics = defaultdict(lambda: {'val': 0, 'count': 0, 'avg': 0})

    def update(self, metric_name, val):
      metric = self.metrics[metric_name]

      metric['val'] += val
      metric['count'] += 1
      metric['avg'] = metric['val'] / metric['count']

    def __str__(self):
      return ' | '.join(
          [
              f'{metric_name}: {format(metric["avg"], f".{self.float_precision}f")}'
              for (metric_name, metric) in self.metrics.items()
          ]
      )

In [11]:
def output_to_prediction(net_output):
    probabilities = nn.Softmax(dim=1)(net_output)
    pred = torch.argmax(probabilities, dim=1)

    return pred

In [12]:
class ModelTrainAndVal:
  def __init__(self, model, params, train_dataset, val_dataset, model_name):
    self.params = params

    self.model = model.to(self.params['device'])

    self.train_dataset = train_dataset
    self.val_dataset = val_dataset

    self.model_name = model_name
    self.train_metrics = []
    self.val_metrics = []

  def train_epoch(self, train_loader, epoch, metric_monitor, criterion, optimizer):
    metric_monitor.reset()
    self.model.train()
    stream = tqdm(train_loader)
    for i, (images, target) in enumerate(stream, start=1):
        images = images.float()

        images = images.to(self.params['device'], non_blocking=True)
        target = target.to(self.params['device'], non_blocking=True)

        output = self.model(images)

        loss = criterion(output, target)
        metric_monitor.update('Loss', loss.item())

        with torch.no_grad():
          pred = output_to_prediction(output)
          correct = (pred == target).float().sum() / self.params["batch_size"]

          metric_monitor.update('Accuracy', correct.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        stream.set_description(
            f'Epoch: {epoch}. Train.    {metric_monitor}'
        )

  def validate_epoch(self, val_loader, epoch, metric_monitor, criterion):
    metric_monitor.reset()
    self.model.eval()
    stream = tqdm(val_loader)
    with torch.no_grad():
      correct = 0.0
      for i, (images, target) in enumerate(stream, start=1):
        images = images.float()

        images = images.to(self.params['device'], non_blocking=True)
        target = target.to(self.params['device'], non_blocking=True)

        output = self.model(images)

        loss = criterion(output, target)
        metric_monitor.update('Loss', loss.item())

        with torch.no_grad():
          pred = output_to_prediction(output)
          correct = (pred == target).float().sum() / self.params["batch_size"]

          metric_monitor.update('Accuracy', correct.item())

        stream.set_description(
            f'Epoch: {epoch}. Validate. {metric_monitor}'
        )

  def train(self, epoch_num):
    train_loader = DataLoader(
        self.train_dataset,
        batch_size=self.params['batch_size'],
        num_workers=2,
        pin_memory=True
    )
    val_loader = DataLoader(
        self.val_dataset,
        batch_size=self.params['batch_size'],
        pin_memory=True
    )

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(self.model.parameters(), lr=self.params['learning_rate'])

    saved_num = 0
    for epoch in range(1, epoch_num+1):
      train_metric_monitor = MetricMonitor()
      self.train_epoch(train_loader, epoch, train_metric_monitor, criterion, optimizer)
      self.train_metrics.append(train_metric_monitor.metrics)

      val_metric_monitor = MetricMonitor()
      self.validate_epoch(val_loader, epoch, val_metric_monitor, criterion)
      self.val_metrics.append(val_metric_monitor.metrics)

      # if val_metric_monitor.metrics['Accuracy']['avg'] >= self.SAVE_THRESHOLD and saved_num <= self.LIMIT_SAVED:
      #   saved_num += 1
      #   torch.save(
      #       self.model.state_dict(),
      #       f'{self.PATH_MODELS_SAVES}{self.model_name}-e{epoch}' +
      #       f'-v_acc{val_metric_monitor.metrics["Accuracy"]["avg"]}' +
      #       f'-t_acc{train_metric_monitor.metrics["Accuracy"]["avg"]}'
      #   )

    return self.model

In [13]:
model = models.squeezenet1_1(pretrained=True)

In [14]:
model.classifier[1] = torch.nn.Conv2d(in_channels=512, out_channels=len(BrickDataset.ALL_LABELS), kernel_size=(1, 1), stride=(1, 1))

In [15]:
params = {
    'learning_rate': 3.e-4,
    'batch_size': 32,
    'device': 'cuda'
}

In [16]:
model_train_and_val = ModelTrainAndVal(model, params, train_dataset, val_dataset, 'squeezenet')

In [17]:
model = model_train_and_val.train(10)

Epoch: 1. Train.    Loss: 1.3889 | Accuracy: 0.5293: 100%|██████████| 16/16 [00:04<00:00,  3.41it/s]
Epoch: 1. Validate. Loss: 0.1597 | Accuracy: 0.9420: 100%|██████████| 7/7 [00:01<00:00,  5.87it/s]
Epoch: 2. Train.    Loss: 0.1249 | Accuracy: 0.9512: 100%|██████████| 16/16 [00:03<00:00,  4.70it/s]
Epoch: 2. Validate. Loss: 0.2296 | Accuracy: 0.8750: 100%|██████████| 7/7 [00:01<00:00,  5.99it/s]
Epoch: 3. Train.    Loss: 0.0668 | Accuracy: 0.9688: 100%|██████████| 16/16 [00:03<00:00,  4.67it/s]
Epoch: 3. Validate. Loss: 0.0228 | Accuracy: 0.9598: 100%|██████████| 7/7 [00:01<00:00,  6.05it/s]
Epoch: 4. Train.    Loss: 0.0452 | Accuracy: 0.9785: 100%|██████████| 16/16 [00:03<00:00,  4.58it/s]
Epoch: 4. Validate. Loss: 0.0238 | Accuracy: 0.9643: 100%|██████████| 7/7 [00:01<00:00,  5.99it/s]
Epoch: 5. Train.    Loss: 0.0159 | Accuracy: 0.9863: 100%|██████████| 16/16 [00:03<00:00,  4.64it/s]
Epoch: 5. Validate. Loss: 0.0007 | Accuracy: 0.9688: 100%|██████████| 7/7 [00:01<00:00,  6.09it/s]


In [18]:
model.eval()

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): MaxPool2d

In [22]:
model = model.to("cpu")

In [24]:
!pwd

/content/drive/MyDrive


In [ ]:
PATH_ONNX_MODEL = 'labs_cv/squeezenet_bricks.onnx'

In [25]:
x = torch.randn(10, 3, 256, 256, requires_grad=True)
torch_out = model(x)

# Export the model
torch.onnx.export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  PATH_ONNX_MODEL,   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

In [30]:
!pip install numpy protobuf==3.16.0
!pip install onnx
!pip install onnxruntime

     |████████████████████████████████| 5.2 MB 4.2 MB/s 


In [31]:
import onnx

onnx_model = onnx.load(PATH_ONNX_MODEL)
onnx.checker.check_model(onnx_model)

In [32]:
import onnxruntime

ort_session = onnxruntime.InferenceSession(PATH_ONNX_MODEL)

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

Exported model has been tested with ONNXRuntime, and the result looks good!
